In [82]:
import numpy as np

# Create environment

In [83]:
number_of_states = 16
number_of_terminal_states = 2
number_of_non_terminal_states = number_of_states - number_of_terminal_states

In [84]:
max_number_of_actions = 4

In [85]:
number_of_actions_per_non_terminal_state = np.repeat(a = max_number_of_actions, repeats = number_of_non_terminal_states)

In [86]:
number_of_state_action_successor_states = np.repeat(a = 1, repeats = number_of_states * max_number_of_actions)

In [87]:
number_of_state_action_successor_states = np.reshape(a = number_of_state_action_successor_states, newshape = (number_of_states, max_number_of_actions))

In [88]:
state_action_successor_state_indices = np.array([1, 0, 14, 4, 2, 1, 0, 5, 2, 2, 1, 6, 4, 14, 3, 7, 5, 0, 3, 8, 6, 1, 4, 9, 6, 2, 5, 10, 8, 3, 7, 11, 9, 4, 7, 12, 10, 5, 8, 13, 10, 6, 9, 15, 12, 7, 11, 11, 13, 8, 11, 12, 15, 9, 12, 13], dtype = np.int64)

In [89]:
state_action_successor_state_transition_probabilities = np.repeat(a = 1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [90]:
state_action_successor_state_rewards = np.repeat(a = -1.0, repeats = number_of_non_terminal_states * max_number_of_actions * 1)

In [91]:
state_action_successor_state_indices = np.reshape(a = state_action_successor_state_indices, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_transition_probabilities = np.reshape(a = state_action_successor_state_transition_probabilities, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))
state_action_successor_state_rewards = np.reshape(a = state_action_successor_state_rewards, newshape = (number_of_non_terminal_states, max_number_of_actions, 1))

# Set hyperparameters

In [92]:
# Set the number of episodes
number_of_episodes = 10000
# Set the maximum episode length
maximum_episode_length = 200
# Set learning rate alpha
alpha = 0.1
# Set epsilon for our epsilon level of exploration
epsilon = 0.1
# Set discounting factor gamma
discounting_factor_gamma = 1.0

# Create value function and policy arrays

In [93]:
state_action_value_function = np.repeat(a = 0.0, repeats = number_of_states * max_number_of_actions)
state_action_value_function = np.reshape(a = state_action_value_function, newshape = (number_of_states, max_number_of_actions))

In [94]:
policy = np.repeat(a = 1.0 / max_number_of_actions, repeats = number_of_non_terminal_states * max_number_of_actions)
policy = np.reshape(a = policy, newshape = (number_of_non_terminal_states, max_number_of_actions))

# Create algorithm

In [95]:
# Set random seed so that everything is reproducible
np.random.seed(seed = 0)

In [96]:
# This function initializes episodes
def initialize_epsiode(number_of_non_terminal_states):
    step_count = 0

    # Initial state
    initial_state_index = np.random.randint(low = 0, high = number_of_non_terminal_states, dtype = np.int64) # randomly choose an initial state from all non-terminal states

    return initial_state_index

In [97]:
# This function selects a policy greedily from the state-action-value function
def epsilon_greedy_policy_from_state_action_function(max_number_of_actions, state_action_value_function, epsilon, state_index, policy):
    # Save max state-action value and find the number of actions that have the same max state-action value
    max_action_value = np.max(a = state_action_value_function[state_index, :])
    max_action_count = np.count_nonzero(a = state_action_value_function[state_index, :] == max_action_value)

    # Apportion policy probability across ties equally for state-action pairs that have the same value and zero otherwise
    if max_action_count == max_number_of_actions:
        max_policy_apportioned_probability_per_action = 1.0 / max_action_count
        remaining_apportioned_probability_per_action = 0.0
    else:
        max_policy_apportioned_probability_per_action = (1.0 - epsilon) / max_action_count
        remaining_apportioned_probability_per_action = epsilon / (max_number_of_actions - max_action_count)

    policy[state_index, :] = np.where(state_action_value_function[state_index, :] == max_action_value, max_policy_apportioned_probability_per_action, remaining_apportioned_probability_per_action)

    return policy

In [98]:
# This function loops through episodes and updates the policy
def loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length, state_index):
    expected_return = 0.0
    weight = 1.0
    
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Choose policy for chosen state by epsilon-greedy choosing from the state-action-value function
        policy = epsilon_greedy_policy_from_state_action_function(max_number_of_actions, state_action_value_function, epsilon, state_index, policy)

        # Get epsilon-greedy action
        action_index = np.random.choice(a = max_number_of_actions, p = policy[state_index, :])
        
        # Get reward
        successor_state_transition_index = np.random.choice(a = number_of_state_action_successor_states[state_index, action_index], p = state_action_successor_state_transition_probabilities[state_index, action_index, :])

        reward = state_action_successor_state_rewards[state_index, action_index, successor_state_transition_index]

        # Get next state
        next_state_index = state_action_successor_state_indices[state_index, action_index, successor_state_transition_index]

        # Check to see if we actioned into a terminal state
        if next_state_index >= number_of_non_terminal_states:
            state_action_value_function[state_index, action_index] += alpha * (reward - state_action_value_function[state_index, action_index])
            break; # episode terminated since we ended up in a terminal state
        else:
            # Get next action, using expectation value
            state_value_function_expected_value_on_policy = np.sum(a = policy[next_state_index, :] * state_action_value_function[next_state_index, :])

            # Calculate state-action-function expectation
            state_action_value_function[state_index, action_index] += alpha * (reward + discounting_factor_gamma * state_value_function_expected_value_on_policy - state_action_value_function[state_index, action_index])

            # Update state to next state
            state_index = next_state_index

    return state_action_value_function, policy

In [99]:
def on_policy_temporal_difference_expected_sarsa(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length):
    for episode in range(0, number_of_episodes):
        # Initialize episode to get initial state
        initial_state_index = initialize_epsiode(number_of_non_terminal_states)

        # Loop through episode and update the policy
        state_action_value_function, policy = loop_through_episode(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length, initial_state_index)
    
    return state_action_value_function, policy

# Run algorithm

In [100]:
# Print initial arrays
print("\nInitial state-action value function")
print(state_action_value_function)

print("\nInitial policy")
print(policy)

# Run on policy temporal difference expected sarsa
state_action_value_function, policy = on_policy_temporal_difference_expected_sarsa(number_of_non_terminal_states, max_number_of_actions, number_of_state_action_successor_states, state_action_successor_state_indices, state_action_successor_state_transition_probabilities, state_action_successor_state_rewards, state_action_value_function, policy, alpha, epsilon, discounting_factor_gamma, maximum_episode_length)

# Print final results
print("\nFinal state-action value function")
print(state_action_value_function)

print("\nFinal policy")
print(policy)


Initial state-action value function
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state-action value function
[[-3.37287365 -2.19625563 -1.         -3.33369259]
 [-4.37617941 -3.35755874 -2.19673315 -4.30122993]
 [-4.36370404 -4.16984841 -3.37773859 -3.3777405 ]
 [-3.33314617 -1.         -2.19593578 -3.3723879 ]
 [-4.33003665 -2.19670422 -2.19670106 -4.27179421]
 [-3.35281672 -3.34921245 -3.33655474 -3.33654761]
 [-3.35997858 -4.38409463 -4.29613151 -2.1